In [15]:
# This code was written in TF 1.12 but should be supported all the way through
# TF 1.15. Untested in TF 2.0+.
#%tensorflow_version 1.x

# Download the raw data (only 108 epoch data points, for full dataset,
# uncomment the second line for nasbench_full.tfrecord).

!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# !curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
!pip install ./nasbench

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).
from nasbench import api

# Use nasbench_full.tfrecord for full dataset (run download command above).
nasbench = api.NASBench('nasbench_only108.tfrecord')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  498M    0 1086k    0     0  1086k      0  0:07:49  0:00:01  0:07:48  827k
  2  498M    2 11.0M    0     0  5656k      0  0:01:30  0:00:02  0:01:28 4957k
  4  498M    4 22.9M    0     0  7824k      0  0:01:05  0:00:03  0:01:02 7151k
  6  498M    6 33.6M    0     0  8624k      0  0:00:59  0:00:04  0:00:55 8056k
  9  498M    9 45.0M    0     0  9235k      0  0:00:55  0:00:05  0:00:50 9351k
 11  498M   11 55.8M    0     0  9528k      0  0:00:53  0:00:06  0:00:47 11.0M
 13  498M   13 65.4M    0     0  9570k      0  0:00:53  0:00:07  0:00:46 10.8M
 15  498M   15 76.7M    0     0  9828k      0  0:00:51  0:00:08  0:00:43 10.7M
 17  498M   17 87.4M    0     0  9949k      0  0:00

ModuleNotFoundError: No module named 'absl'

In [30]:
#to set up map_elites
!git clone https://github.com/resibots/pymap_elites
!pip install ./pymap_elites

from pymap_elites import setup

AttributeError: 'list' object has no attribute 'split'

In [31]:
# Standard imports
import copy
import numpy as np
import matplotlib.pyplot as plt
import random
import math

import multiprocessing
from pathlib import Path
import sys
from collections import defaultdict
#from sklearn.cluster import KMeans

import argparse
import collections
import os
import json
from copy import deepcopy
#import ConfigSpace
from map_elites import common as cm

#from tabular_benchmarks import FCNetProteinStructureBenchmark, FCNetSliceLocalizationBenchmark,\
#    FCNetNavalPropulsionBenchmark, FCNetParkinsonsTelemonitoringBenchmark
#from tabular_benchmarks import NASCifar10A, NASCifar10B


# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

ModuleNotFoundError: No module named 'map_elites'

In [5]:
default_params = \
    {
        # more of this -> higher-quality CVT
        "cvt_samples": 25000,
        # we evaluate in batches to paralleliez
        "batch_size": 100,
        # proportion of niches to be filled before starting
        "random_init": 0.1,
        # batch for random initialization
        "random_init_batch": 100,
        # when to write results (one generation = one batch)
        "dump_period": 10000,
        # do we use several cores?
        "parallel": True,
        # do we cache the result of CVT and reuse?
        "cvt_use_cache": True,
        # min/max of parameters
        "min": 0,
        "max": 1,
        # only useful if you use the 'iso_dd' variation operator
        "iso_sigma": 0.01,
        "line_sigma": 0.2
    }

In [24]:
class Spacies:
    def __init__(self, x, desc, fitness, centroid = None):
        #x: solution
        #desc: description
        #fitness: fitness function
        self.x = x
        self.desc = desc
        self.fitness = fitness
        self.centroid = centroid
        
    def polynomial_mutation(x):
        y = x.copy()
        eta_m = 5.0;
        r = np.random.random(size=len(x))
        for i in range(0, len(x)):
            if r[i] < 0.5:
                delta_i = math.pow(2.0 * r[i], 1.0 / (eta_m + 1.0)) - 1.0
            else:
                delta_i = 1 - math.pow(2.0 * (1.0 - r[i]), 1.0 / (eta_m + 1.0))
            y[i] += delta_i
        return y

    def sbx(x, y, params):
        eta = 10.0
        xl = params['min']
        xu = params['max']
        z = x.copy()
        r1 = np.random.random(size=len(x))
        r2 = np.random.random(size=len(x))
    
        for i in range(0, len(x)):
            if abs(x[i] - y[i]) > 1e-15:
                x1 = min(x[i], y[i])
                x2 = max(x[i], y[i])
    
                beta = 1.0 + (2.0 * (x1 - xl) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                rand = r1[i]
                if rand <= 1.0 / alpha:
                    beta_q = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    beta_q = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))
    
                c1 = 0.5 * (x1 + x2 - beta_q * (x2 - x1))
    
                beta = 1.0 + (2.0 * (xu - x2) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                if rand <= 1.0 / alpha:
                    beta_q = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    beta_q = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))
                c2 = 0.5 * (x1 + x2 + beta_q * (x2 - x1))
    
                c1 = min(max(c1, xl), xu)
                c2 = min(max(c2, xl), xu)
    
                if r2[i] <= 0.5:
                    z[i] = c2
                else:
                    z[i] = c1
        return z
    
    
    def iso_dd(x, y, params):
        assert(x.shape == y.shape)
        p_max = np.array(params["max"])
        p_min = np.array(params["min"])
        a = np.random.normal(0, params['iso_sigma'], size=len(x))
        b = np.random.normal(0, params['line_sigma'])
        norm = np.linalg.norm(x - y)
        z = x.copy() + a + b * (x - y)
        return np.clip(z, p_min, p_max)
    
    
    def variation(x, z, params):
        assert(x.shape == z.shape)
        y = sbx(x, z, params)
        return y
    
    def __centroids_filename(k, dim):
        return 'centroids_' + str(k) + '_' + str(dim) + '.dat'
    
    
    def __write_centroids(centroids):
        k = centroids.shape[0]
        dim = centroids.shape[1]
        filename = __centroids_filename(k, dim)
        with open(filename, 'w') as f:
            for p in centroids:
                for item in p:
                    f.write(str(item) + ' ')
                f.write('\n')

    
    def cvt(k, dim, samples, cvt_use_cache=True):
        # check if we have cached values
        fname = __centroids_filename(k, dim)
        if cvt_use_cache:
            if Path(fname).is_file():
                print("WARNING: using cached CVT:", fname)
                return np.loadtxt(fname)
        # otherwise, compute cvt
        print("Computing CVT (this can take a while...):", fname)
    
        x = np.random.rand(samples, dim)
        k_means = KMeans(init='k-means++', n_clusters=k,
                         n_init=1, n_jobs=-1, verbose=1)#,algorithm="full")
        k_means.fit(x)
        __write_centroids(k_means.cluster_centers_)
    
        return k_means.cluster_centers_
    
    
    def make_hashable(array):
        return tuple(map(float, array))
    
    
    def parallel_eval(evaluate_function, to_evaluate, pool, params):
        if params['parallel'] == True:
            s_list = pool.map(evaluate_function, to_evaluate)
        else:
            s_list = map(evaluate_function, to_evaluate)
        return list(s_list)
    
    # format: fitness, centroid, desc, genome \n
    # fitness, centroid, desc and x are vectors
    def __save_archive(archive, gen):
        def write_array(a, f):
            for i in a:
                f.write(str(i) + ' ')
        filename = 'archive_' + str(gen) + '.dat'
        with open(filename, 'w') as f:
            for k in archive.values():
                f.write(str(k.fitness) + ' ')
                write_array(k.centroid, f)
                write_array(k.desc, f)
                write_array(k.x, f)
                f.write("\n")  

    # map-elites algorithm (CVT variant)
def compute(dim_map, dim_x, f,
            n_niches=1000,
            max_evals=1e5,
            params=cm.default_params,
            log_file=None,
            variation_operator=cm.variation):
    """CVT MAP-Elites
       Vassiliades V, Chatzilygeroudis K, Mouret JB. Using centroidal voronoi tessellations to scale up the multidimensional archive of phenotypic elites algorithm. IEEE Transactions on Evolutionary Computation. 2017 Aug 3;22(4):623-30.
       Format of the logfile: evals archive_size max mean median 5%_percentile, 95%_percentile
    """
    # setup the parallel processing pool
    num_cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(num_cores)

    # create the CVT
    c = cm.cvt(n_niches, dim_map,
              params['cvt_samples'], params['cvt_use_cache'])
    kdt = KDTree(c, leaf_size=30, metric='euclidean')
    cm.__write_centroids(c)

    archive = {} # init archive (empty)
    n_evals = 0 # number of evaluations since the beginning
    b_evals = 0 # number evaluation since the last dump

    # main loop
    while (n_evals < max_evals):
        to_evaluate = []
        # random initialization
        if len(archive) <= params['random_init'] * n_niches:
            for i in range(0, params['random_init_batch']):
                x = np.random.uniform(low=params['min'], high=params['max'], size=dim_x)
                to_evaluate += [(x, f)]
        else:  # variation/selection loop
            keys = list(archive.keys())
            # we select all the parents at the same time because randint is slow
            rand1 = np.random.randint(len(keys), size=params['batch_size'])
            rand2 = np.random.randint(len(keys), size=params['batch_size'])
            for n in range(0, params['batch_size']):
                # parent selection
                x = archive[keys[rand1[n]]]
                y = archive[keys[rand2[n]]]
                # copy & add variation
                z = variation_operator(x.x, y.x, params)
                to_evaluate += [(z, f)]
        # evaluation of the fitness for to_evaluate
        s_list = cm.parallel_eval(__evaluate, to_evaluate, pool, params)
        # natural selection
        for s in s_list:
            __add_to_archive(s, s.desc, archive, kdt)
        # count evals
        n_evals += len(to_evaluate)
        b_evals += len(to_evaluate)

        # write archive
        if b_evals >= params['dump_period'] and params['dump_period'] != -1:
            print("[{}/{}]".format(n_evals, int(max_evals)), end=" ", flush=True)
            cm.__save_archive(archive, n_evals)
            b_evals = 0
        # write log
        if log_file != None:
            fit_list = np.array([x.fitness for x in archive.values()])
            log_file.write("{} {} {} {} {} {} {}\n".format(n_evals, len(archive.keys()),
                    fit_list.max(), np.mean(fit_list), np.median(fit_list),
                    np.percentile(fit_list, 5), np.percentile(fit_list, 95)))
            log_file.flush()
    cm.__save_archive(archive, n_evals)
    return archive

NameError: name 'cm' is not defined

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('--run_id', default=0, type=int, nargs='?', help='unique number to identify this run')
parser.add_argument('--benchmark', default="protein_structure", type=str, nargs='?', help='specifies the benchmark')
parser.add_argument('--n_iters', default=100, type=int, nargs='?', help='number of iterations for optimization method')
parser.add_argument('--output_path', default="./", type=str, nargs='?',
                    help='specifies the path where the results will be saved')
parser.add_argument('--data_dir', default="./", type=str, nargs='?', help='specifies the path to the tabular data')
parser.add_argument('--pop_size', default=100, type=int, nargs='?', help='population size')
parser.add_argument('--sample_size', default=10, type=int, nargs='?', help='sample_size')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--run_id [RUN_ID]]
                             [--benchmark [BENCHMARK]] [--n_iters [N_ITERS]]
                             [--output_path [OUTPUT_PATH]]
                             [--data_dir [DATA_DIR]] [--pop_size [POP_SIZE]]
                             [--sample_size [SAMPLE_SIZE]]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\LG\AppData\Roaming\jupyter\runtime\kernel-9a81f7fe-3149-41db-88e7-eec6624d4d8b.json


SystemExit: 2

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
if args.benchmark == "nas_cifar10a":
    b = NASCifar10A(data_dir=args.data_dir)

elif args.benchmark == "nas_cifar10b":
    b = NASCifar10B(data_dir=args.data_dir)

elif args.benchmark == "protein_structure":
    b = FCNetProteinStructureBenchmark(data_dir=args.data_dir)

elif args.benchmark == "slice_localization":
    b = FCNetSliceLocalizationBenchmark(data_dir=args.data_dir)

elif args.benchmark == "naval_propulsion":
    b = FCNetNavalPropulsionBenchmark(data_dir=args.data_dir)

elif args.benchmark == "parkinsons_telemonitoring":
    b = FCNetParkinsonsTelemonitoringBenchmark(data_dir=args.data_dir)

output_path = os.path.join(args.output_path, "regularized_evolution")
os.makedirs(os.path.join(output_path), exist_ok=True)

cs = b.get_configuration_space()

history = regularized_evolution(
    cycles=args.n_iters, population_size=args.pop_size, sample_size=args.sample_size)

if args.benchmark == "nas_cifar10a" or args.benchmark == "nas_cifar10b":
    res = b.get_results(ignore_invalid_configs=True)
else:
    res = b.get_results()

fh = open(os.path.join(output_path, 'run_%d.json' % args.run_id), 'w')
json.dump(res, fh)
fh.close()